<a href="https://colab.research.google.com/github/GirolamoOddo/AppliedMath_Notebooks/blob/main/AdversarialSearch_for_ZeroSumScenario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



###__0. Overview:__  

This notebook implements a simple two-player game where one is the user and the other is a minmax algorithm, this is intended to give a practical insight into the topic of adversarial research.

The game, in particular, involves two players: an attacker and a defender.
Each player takes turns to either exploit or patch vulnerabilities.  

The vulnerabilities have risk ranks ranging from 0 to 5.  
After being patched or exploited, a vulnerability enters a cooldown period during which it cannot be targeted again.  
The game ends when one player accumulates a score of 4, indicating a significant success in either exploitation or patching.  

###__1. Components of the Code:__
####Game Logic Functions:  
__play_round(state, cooldown, player_turn):__   
>Manages the flow of a single round in the game.  
attacker_move(state, cooldown, attacker_score, defender_score): Determines the attacker's move using the Minimax algorithm.  
defender_move(state, cooldown, attacker_score, defender_score): Allows the defender to choose a vulnerability to patch.  
generate_random_initial_state(): Initializes the game state with random vulnerability ranks.  

####Minimax Algorithm:  
__min_max()__  
>This function implements the Minimax algorithm with alpha-beta pruning to determine the best move for the attacker.  
It evaluates possible future states of the game by recursively exploring the game tree up to a certain depth.  
The algorithm alternates between maximizing the attacker's advantage and minimizing the defender's advantage, assuming both players play optimally.  
The evaluate_state() function provides a heuristic evaluation of a game state based on the attacker's advantage.  

####Main Functionality:  
__main()__  
>This function initializes the game, provides instructions, and orchestrates the gameplay between the attacker and defender.  
It prints the game state, prompts players for their moves, and updates the game accordingly until one player wins or the game ends in a draw.    

>  

###__2. Introduction to Minimax Algorithm and Adversarial Search:__  

####Adversarial Search:
__Problem Space:__
>Adversarial search deals with problems where multiple competing agents make decisions in an adversarial environment.  
These problems are typically modeled as game trees, where each agent's actions affect the outcome of the game.    

__Perfect Information Games:__  
>Adversarial search is commonly applied to games with perfect information, where all players have complete knowledge of the game state.  
This includes classic board games like chess and Go.  

__Strategic Decision Making:__
>The goal of adversarial search is to develop strategies that lead to favorable outcomes, even in the face of opposition from other agents.  
It involves exploring the space of possible actions and anticipating the moves of opponents to make informed decisions.  

__Algorithmic Approaches:__
>Various algorithms are used in adversarial search, including Minimax, Monte Carlo Tree Search (MCTS), and reinforcement learning techniques like Deep Q-Networks (DQN).  
Each algorithm has its strengths and weaknesses, and the choice depends on factors such as the complexity of the game, the availability of computational resources, and the desired level of performance.

####Minimax Algorithm:

__Decision Theory Foundation:__  
>The Minimax algorithm originates from decision theory, aiming to minimize the maximum possible loss (hence "minimax") in a worst-case scenario.  
In game theory, it's particularly relevant for zero-sum games where the gain of one player directly corresponds to the loss of the other.    

__Recursive Search:__  
>Minimax employs a recursive search through the game tree, where each level represents a player's turn and each node represents a possible game state.  
At the leaf nodes, the game state is evaluated using a heuristic function, providing an estimate of the desirability of that state for the current player.  

__Maximization and Minimization:__  
>The algorithm alternates between two types of nodes in the tree: maximizing nodes (for the player seeking to maximize their advantage) and minimizing nodes (for the opponent seeking to minimize that advantage).
At maximizing nodes, the algorithm selects the child node with the highest heuristic value, representing the best move for the current player.
At minimizing nodes, the algorithm selects the child node with the lowest heuristic value, representing the best countermove for the opponent.  


---

#__Now run the cell below and play against MinMax!__



In [12]:
import random

def evaluate_state(state, attacker_score, defender_score):
    attacker_advantage = attacker_score - defender_score
    attacker_winning = +10 if attacker_score == 4 else 0
    defender_winning = -10 if defender_score == 4 else 0
    return round(attacker_advantage, 4) + attacker_winning + defender_winning

def min_max(state, cooldown, attacker_score, defender_score, depth, player, alpha, beta):
    if depth == 0 or attacker_score >= 4 or defender_score >= 4:
      return None, evaluate_state(state, attacker_score, defender_score), attacker_score, defender_score

    cooldown_sim = cooldown.copy()
    if depth != 5: #  avoiding first minmax call, 5 is the initial depth value
       for i in range(len(cooldown_sim)):
           cooldown_sim[i] = max(0, cooldown_sim[i] - 1)

    best_move = None
    if player == 'attacker':
        best_score = float('-inf')
        for i in range(len(state)):
            if state[i] > 0 and state[i] < 5 and cooldown_sim[i] == 0:
                state[i] += 1
                cooldown_sim[i] = 4
                attacker_score = state.count(5)

                _, score, _, _ = min_max(state, cooldown_sim, attacker_score, defender_score, depth - 1, 'defender', alpha, beta)

                state[i] -= 1
                cooldown_sim[i] = 0
                attacker_score = state.count(5)
                if score > best_score:
                    best_score = score
                    best_move = i
                alpha = max(alpha, score)
                #if beta <= alpha:
                #    break
        return best_move, best_score, attacker_score, defender_score
    else:
        best_score = float('inf')
        for i in range(len(state)):
            if state[i] > 0 and state[i] < 5 and cooldown_sim[i] == 0:
                state[i] -= 1
                cooldown_sim[i] = 6
                defender_score = state.count(0)

                _, score, _, _ = min_max(state, cooldown_sim, attacker_score, defender_score, depth - 1, 'attacker', alpha, beta)

                state[i] += 1
                cooldown_sim[i] = 0
                defender_score = state.count(0)
                if score < best_score:
                    best_score = score
                    best_move = i
                beta = min(beta, score)
                #if beta <= alpha:
                #    break
        return best_move, best_score, attacker_score, defender_score

def attacker_move(state, cooldown, attacker_score, defender_score, depth=5):
    alpha = float('-inf')
    beta  = float(' inf')
    best_move, best_score, attacker_score, defender_score = min_max(state, cooldown, attacker_score, defender_score, depth, 'attacker', alpha, beta)

    if best_move is not None:
        print(f"MinMax best score for depth {depth}: {best_score}")
        print(f"Attacker winning pattern found") if best_score >= float(' 4') else None
        print(f"Defender winning pattern found") if best_score <= float('-4') else None
        print(f"No valid moves situation for defender found") if best_score == float('inf') else None
        return best_move, state[best_move], attacker_score, defender_score
    else:
        print(f"MinMax best score for depth {depth}: {best_score}")
        print(f"No valid moves situation for attacker found")
        # Choose a suboptimal move by selecting the first valid move
        for i in range(len(state)):
            if state[i] > 0 and state[i] < 5 and cooldown[i] == 0:
               return i, state[i], attacker_score, defender_score

        # If no valid move is available, return None for move and unchanged state
        return None, None, attacker_score, defender_score

def defender_move(state, cooldown, attacker_score, defender_score):
    valid_moves = [i for i in range(len(state)) if state[i] > 0 and state[i] < 5 and cooldown[i] == 0]
    if not valid_moves:
        return None
    while True:
        move = int(input("Enter the vulnerability you want to patch (0-9): "))
        if move in valid_moves:
            return move, state[move], attacker_score, defender_score + state.count(0)
        else:
            print("Invalid move! Please choose a vulnerability with a risk value between 1 and 4 and cooldown equal to 0.")

def play_round(state, cooldown, player_turn):
    print('-------------------------------------------------------------')
    attacker_score = state.count(5)  # Update attacker's score
    defender_score = state.count(0)  # Update defender's score
    print("Attacker's Score:", attacker_score)
    print("Defender's Score:", defender_score)
    print("Vulnerability   :", [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
    print("Current state   :", state)
    print("Current cooldown:", cooldown)

    if attacker_score >= 4:
        print("Attacker wins!")
        return True
    elif defender_score >= 4:
        print("Defender wins!")
        return True

    if player_turn:
        print("Your turn...")
        move = defender_move(state, cooldown, attacker_score, defender_score)
        if move is None:
            print("No valid move for defender. Attacker wins!")
            return True
        print("Defender patches vulnerability:", move[0])
        state[move[0]] -= 1
        cooldown[move[0]] = 6

    else:
        print("Attacker's turn...")
        move = attacker_move(state, cooldown, attacker_score, defender_score)
        # Check if move is not None before accessing its elements
        if move[0] is None:
            print("No valid move for attacker. Defender wins!")
            return True
        elif move[0] is not None:
            print("Attacker exploits vulnerability:", move[0])
            state[move[0]] += 1
            cooldown[move[0]] = 4

    for i in range(len(cooldown)):
         cooldown[i] = max(0, cooldown[i] - 1)

    return False

def generate_random_initial_state():
    return [random.randint(2, 3) for _ in range(10)] # debug fixed start [0,5,0,4,0,1,4,5,3,3]

#-------------------------------------------------------------------------------

def main():
    print("Welcome to the Defender-Attacker game!")
    print("Rules:")
    print("- The game is played between a defender and an attacker.")
    print("- The defender's goal is to patch vulnerabilities.")
    print("- The attacker's goal is to exploit vulnerabilities.")
    print("- Each vulnerability has a risk rank from 0 to 5.")
    print("- The defender patches vulnerabilities by reducing their risk rank by 1.")
    print("- The attacker exploits vulnerabilities by increasing their risk rank by 1.")
    print("- After being patched, a vulnerability cannot be patched/exploited again for 5 turns.")
    print("- After being exploited, a vulnerability cannot be patched/exploited again for 3 turns.")
    print("- The game ends when one player reaches a score of 4.")
    print()

    initial_state = generate_random_initial_state()
    cooldown = [0] * len(initial_state)

    player_turn = True

    print('-------------------------------------------------------------')
    print('> MinMax Algorithm Play as Attacker!')
    print('Note: MinMax best score is the Euristic used for the search')
    print('      MinMax score is computed as Attacker_scr - Defender_scr')
    print('      if best_score >  4, means that an attacker winning was found')
    print('      if best_score < -4, means that a  defender winning was found')
    print()
    print('> You Play as Defender!')
    print('-------------------------------------------------------------')

    round = 0
    while True:
        print()
        print('Round:', int(round))
        round += 0.5
        if play_round(initial_state, cooldown, player_turn):
            break

        if all(cd == 1 for cd in cooldown):
            cooldown = [0] * len(initial_state)
        player_turn = not player_turn


if __name__ == "__main__":
    main()


Welcome to the Defender-Attacker game!
Rules:
- The game is played between a defender and an attacker.
- The defender's goal is to patch vulnerabilities.
- The attacker's goal is to exploit vulnerabilities.
- Each vulnerability has a risk rank from 0 to 5.
- The defender patches vulnerabilities by reducing their risk rank by 1.
- The attacker exploits vulnerabilities by increasing their risk rank by 1.
- After being patched, a vulnerability cannot be patched/exploited again for 5 turns.
- After being exploited, a vulnerability cannot be patched/exploited again for 3 turns.
- The game ends when one player reaches a score of 4.

-------------------------------------------------------------
> MinMax Algorithm Play as Attacker!
Note: MinMax best score is the Euristic used for the search
      MinMax score is computed as Attacker_scr - Defender_scr
      if best_score >  4, means that an attacker winning was found
      if best_score < -4, means that a  defender winning was found

> You Pla